In [1]:
# Importamos las librerías con las que vamos a trabajar
import pandas as pd
import os
import gc

## Reviews California

En primer lugar, vamos a transformar la información correspondiente a las **reviews de Google Maps** del estado de **California**, que se encuentra repartida en 18 archivos JSON:

In [2]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "Datasets/California"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
reviews = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        reviews.append(df)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_reviews = pd.concat(reviews, ignore_index=True)

# Se chequean los primeros 5 valores:
df_reviews.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [3]:
# Se exporta el archivo en formato parquet:
df_reviews.to_parquet('reviews_california.parquet', engine="pyarrow")

In [3]:
# Se buscan los valores únicos en gmap_id:
df_reviews = df_reviews.dropna(subset="gmap_id")
id_unicos = df_reviews["gmap_id"].unique()
id_unicos

array(['0x80c2c778e3b73d33:0xbdc58662a4a97d49',
       '0x80c2c632f933b073:0xc31785961fe826a6',
       '0x80c2cf163db6bc89:0x219484e2edbcfa41', ...,
       '0x808fe955b0beae57:0xb3159fe6572670c3',
       '0x80c4bc8ce8524717:0x4a40db0add6a0873',
       '0x80c2bea30829f279:0x39aa953ee93734ed'], dtype=object)

## metadata_sitios

La **"metadata"** corresponde a la **información de** los diferentes **comercios incluidos** en **Google Maps**. La misma se encuentra repartida en 11 archivos JSON, que por cuestiones de procesamiento y almacenamiento se separaron en 3 carpetas:

### Metadata 1

In [3]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "Datasets/metadata1"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
metadata = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df1 = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        metadata.append(df1)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_metadata = pd.concat(metadata, ignore_index=True)

# Se chequean los primeros 5 valores:
df_metadata.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
# Se exporta el archivo en formato parquet:
df_metadata.to_parquet('metadata1.parquet', engine="pyarrow")

In [5]:
gc.collect()

7

### Metadata 2

In [2]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "Datasets/metadata2"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
metadata = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df1 = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        metadata.append(df1)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_metadata2 = pd.concat(metadata, ignore_index=True)

# Se chequean los primeros 5 valores:
df_metadata2.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,"Elite Exteriors Roofing & Restoration, Inc.",None,0x87ccd3297badbb3b:0x18e80a5c9de19f16,None,45.597481,-127.269699,"[Roofing contractor, Construction company, Ser...",4.2,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",None,Closed ⋅ Opens 8AM Mon,"[0x87ccd52d7b36dacd:0x6bb2ca2dd53edcd0, 0x8632...",https://www.google.com/maps/place//data=!4m2!3...
2,Harper's Plumbing,"Harper's Plumbing, 400 S Front St, Gurdon, AR ...",0x86330338d4921e25:0x4af02600facc4465,None,33.918058,-93.155579,[Plumber],5.0,1,None,None,None,None,"[0x86331ef6285a3bf7:0xb56dcfd2525cc4f8, 0x8632...",https://www.google.com/maps/place//data=!4m2!3...
3,Wylie Glass and Salvage Inc,"Wylie Glass and Salvage Inc, 1501 W 3rd St, Ho...",0x86338365e22e037b:0xf2ec124da3ecdfe6,None,33.664719,-93.606783,"[Salvage yard, Auto glass shop, Auto parts sto...",4.3,16,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",{'Health & safety': ['Staff required to disinf...,Closed ⋅ Opens 8AM Mon,"[0x86338340e5d5330b:0x51f8102ff3834ae5, 0x87d2...",https://www.google.com/maps/place//data=!4m2!3...
4,Wholesale Beauty & Convenience,"Wholesale Beauty & Convenience, 1750 Pine St, ...",0x8632dfdb9faed6a3:0x506f0e73cb79c8f2,None,34.123242,-93.071387,"[Convenience store, Cosmetics store]",2.9,8,None,"[[Saturday, 9AM–7PM], [Sunday, Closed], [Monda...","{'Service options': ['In-store shopping', 'Del...",Open ⋅ Closes 7PM,"[0x8632dfe6e3f215cf:0x79d25cac1ba273bc, 0x8632...",https://www.google.com/maps/place//data=!4m2!3...


In [3]:
# Se exporta el archivo en formato parquet:
df_metadata2.to_parquet('metadata2.parquet', engine="pyarrow")

In [3]:
gc.collect()

0

### Metadata 3

In [2]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "Datasets/metadata3"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
metadata = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df1 = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        metadata.append(df1)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_metadata3 = pd.concat(metadata, ignore_index=True)

# Se chequean los primeros 5 valores:
df_metadata3.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,Clean Air Vapors,"Clean Air Vapors, 401 W Main St, Riverton, WY ...",0x8758dd1117f560df:0xe691711d0c88da68,None,43.024439,-108.395705,[Store],4.6,68,None,"[[Saturday, 9AM–7PM], [Sunday, Closed], [Monda...","{'Service options': ['In-store shopping', 'Del...",Open ⋅ Closes 7PM,[0x8758dd1cf86fffe7:0x6bb1a35d921b6caf],https://www.google.com/maps/place//data=!4m2!3...
2,Boot Bar,"Boot Bar, 702 E Main St, Riverton, WY 82501",0x8758dd1d120d3c0d:0xe49a6c2994c781c0,Retail chain with a variety of Western & work-...,43.024925,-108.381859,[Bar],4.3,98,$,"[[Saturday, 10AM–2AM], [Sunday, 12–10PM], [Mon...","{'Service options': ['Outdoor seating', 'Takeo...",Open ⋅ Closes 2AM,"[0x8758dd1b65464237:0x76cc809276a4a18c, 0x8758...",https://www.google.com/maps/place//data=!4m2!3...
3,Boysen Dam,"Boysen Dam, Shoshoni, WY 82649",0x8758af9fcdabbee7:0xefa71efcf509afe8,None,43.416677,-108.177605,[Tourist attraction],4.8,18,None,None,{'Amenities': ['Good for kids']},Open now,None,https://www.google.com/maps/place//data=!4m2!3...
4,Cafe Dacha,"Cafe Dacha, 675 Central Ave Suite 1, Highland ...",0x880fc1dd4696911b:0x1952736e51ae8b7c,None,42.184970,-87.802011,"[Restaurant, Bar, Eastern European restaurant,...",4.8,78,None,"[[Saturday, 4–9PM], [Sunday, 2–8PM], [Monday, ...","{'Service options': ['Curbside pickup', 'No-co...",Closed ⋅ Opens 4PM,"[0x880fc105d4151ba7:0xf62bb081e163d435, 0x880f...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
# Se exporta el archivo en formato parquet:
df_metadata3.to_parquet('metadata3.parquet', engine="pyarrow")

### Metadata completo

Por último, se unen los 3 archivos correspondientes a metadata, y se filtran aquellos que tienen reviews en California:

In [4]:
# Se cargan los 3 datasets creados anteriormente:
meta1 = pd.read_parquet("Datasets/metadata1.parquet")
meta2 = pd.read_parquet("Datasets/metadata2.parquet")
meta3 = pd.read_parquet("Datasets/metadata3.parquet")

# Se concatenan los 3 datasets
metadata_california = pd.concat([meta1, meta2, meta3], ignore_index=True, axis=0)

# Se filtran aquellos incluidos en las reviews de California:
metadata_california = metadata_california[metadata_california["gmap_id"].isin(id_unicos)]
print(metadata_california.shape)
metadata_california.head()

(74620, 15)


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
5,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",0x80c2cf163db6bc89:0x219484e2edbcfa41,None,34.015505,-118.181839,[Fabric store],3.5,6,None,"[[Thursday, 8:30AM–5:30PM], [Friday, 8:30AM–5:...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 5:30PM,"[0x80c2cf042a5d9561:0xd0024ad6f81f1335, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
13,Black Tie Ski Rental Delivery of Mammoth,"Black Tie Ski Rental Delivery of Mammoth, 501 ...",0x80960c29f2e3bf29:0x4b291f0d275a5699,None,37.638754,-118.966055,"[Ski rental service, Snowboard rental service]",5.0,34,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 5PM,"[0x80960dcd6ba76731:0x9a6875ced2f9228e, 0x8096...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
metadata_california.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74620 entries, 2 to 2084965
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              74618 non-null  object 
 1   address           74180 non-null  object 
 2   gmap_id           74620 non-null  object 
 3   description       11189 non-null  object 
 4   latitude          74620 non-null  float64
 5   longitude         74620 non-null  float64
 6   category          74545 non-null  object 
 7   avg_rating        74620 non-null  float64
 8   num_of_reviews    74620 non-null  int64  
 9   price             10944 non-null  object 
 10  hours             64938 non-null  object 
 11  MISC              67593 non-null  object 
 12  state             65648 non-null  object 
 13  relative_results  71048 non-null  object 
 14  url               74620 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 9.1+ MB


In [6]:
# Se resetea el indice:
metadata_california = metadata_california.reset_index()

In [7]:
# Se controlan valores nulos en gmap_id:
metadata_california["gmap_id"].isnull().sum()

0

In [8]:
# Se controlan valores duplicados en gmap_id:
metadata_california["gmap_id"].duplicated().sum()

1215

In [9]:
# Se eliminan valores duplicados de gmap_id:
metadata_california = metadata_california.drop_duplicates(subset="gmap_id").reset_index()
metadata_california.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73405 entries, 0 to 73404
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level_0           73405 non-null  int64  
 1   index             73405 non-null  int64  
 2   name              73403 non-null  object 
 3   address           72978 non-null  object 
 4   gmap_id           73405 non-null  object 
 5   description       11072 non-null  object 
 6   latitude          73405 non-null  float64
 7   longitude         73405 non-null  float64
 8   category          73334 non-null  object 
 9   avg_rating        73405 non-null  float64
 10  num_of_reviews    73405 non-null  int64  
 11  price             10853 non-null  object 
 12  hours             63907 non-null  object 
 13  MISC              66572 non-null  object 
 14  state             64610 non-null  object 
 15  relative_results  69916 non-null  object 
 16  url               73405 non-null  object

In [13]:
# Exportamos la metadata de los locales de california en formato parquet:
metadata_california.to_parquet('metadata_california.parquet', engine="pyarrow")